In [3]:
import numpy as np
import pandas as pd
import glob

# API call to get weather data from Canadian government

https://drive.google.com/drive/folders/1WJCDEU34c60IfOnG4rv5EPZ4IhhW9vZH

Get Daily Data (Must specify month and year)

wget --content-disposition "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=51442&Year=2013&Month=12&Day=14&timeframe=2&submit= Download+Data"

Get Hourly Data for each Day in a Month (Must specify month and year)

wget --content-disposition "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=51442&Year=2013&Month=12&Day=14&timeframe=1&submit= Download+Data"

# Filtering weather data

In [4]:
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
# Concat multiple csv
path = r''
#dailyPath = path+"/daily_2016_2018"
#hourlyPath = path+"hourly_2016_2018/2018"
yearPath = path+"hourly_2016_2018/*"

years = glob.glob(yearPath)

li = []
for year in years:
    all_files = glob.glob(year + "/*.csv")
    #Read all csv file for each year
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, parse_dates = ["Date/Time"])
        #Get hour and minutes from Date/Time
        df["Hour"] = pd.to_datetime(df["Date/Time"]).dt.hour
        df["Minute"] = pd.to_datetime(df["Date/Time"]).dt.minute
        #Select the needed data
        filterDf = df[["Date/Time","Year", "Month", "Day", "Hour", "Minute", "Weather"]]
        #Create a Date column from the Date/Time column
        filterDf["Date"] = filterDf['Date/Time'].apply(lambda x:x.date().strftime('%Y-%m-%d'))
        #Get all the dates in the current month
        filterDf = filterDf.dropna()
        li.append(filterDf)

data = pd.concat(li, axis=0, ignore_index=True)
data

/home/leo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date/Time,Year,Month,Day,Hour,Minute,Weather,Date
0,2018-05-01 01:00:00,2018,5,1,1,0,Mostly Cloudy,2018-05-01
1,2018-05-01 04:00:00,2018,5,1,4,0,Mostly Cloudy,2018-05-01
2,2018-05-01 07:00:00,2018,5,1,7,0,Mostly Cloudy,2018-05-01
3,2018-05-01 10:00:00,2018,5,1,10,0,Mainly Clear,2018-05-01
4,2018-05-01 13:00:00,2018,5,1,13,0,Mainly Clear,2018-05-01
...,...,...,...,...,...,...,...,...
14818,2016-09-30 10:00:00,2016,9,30,10,0,Mostly Cloudy,2016-09-30
14819,2016-09-30 13:00:00,2016,9,30,13,0,Mainly Clear,2016-09-30
14820,2016-09-30 16:00:00,2016,9,30,16,0,Mainly Clear,2016-09-30
14821,2016-09-30 19:00:00,2016,9,30,19,0,Mainly Clear,2016-09-30


# Sort the weather data

In [5]:
data = data.sort_values(by="Date/Time")
data.columns = data.columns.str.upper()
data

,DATE/TIME,YEAR,MONTH,DAY,HOUR,MINUTE,WEATHER,DATE
11161,2016-01-01 01:00:00,2016,1,1,1,0,Clear,2016-01-01
11162,2016-01-01 04:00:00,2016,1,1,4,0,Clear,2016-01-01
11163,2016-01-01 07:00:00,2016,1,1,7,0,Clear,2016-01-01
11164,2016-01-01 10:00:00,2016,1,1,10,0,Clear,2016-01-01
11165,2016-01-01 13:00:00,2016,1,1,13,0,Clear,2016-01-01
...,...,...,...,...,...,...,...,...
6607,2019-10-31 10:00:00,2019,10,31,10,0,Cloudy,2019-10-31
6608,2019-10-31 13:00:00,2019,10,31,13,0,Cloudy,2019-10-31
6609,2019-10-31 16:00:00,2019,10,31,16,0,Mainly Clear,2019-10-31
6610,2019-10-31 19:00:00,2019,10,31,19,0,Mainly Clear,2019-10-31


# Flag the weather condition for each weather

From https://climate.weather.gc.ca/glossary_e.html#weather
<br>
Good weather means:
<br>
- Clear 
- Mainly Clear 
- Mostly Cloudy
- Cloudy 

In [19]:
def weatherCond(x):
    goodCond = ["Clear", "Mainly Clear", "Mostly Cloudy", "Cloudy"]
    if x.strip(" ") in goodCond:
        return "Good"
    return "Bad"
data["WEATHERCOND"] = data["WEATHER"].apply(lambda x: weatherCond(x))

# Write Data to CSV

In [20]:
data.to_csv("cleanedWeatherData.csv",index=False)